In [1]:
from platform import python_version

In [2]:
python_version()

'3.7.5'

In [3]:
# Interactive plotting
%matplotlib qt

import numpy as np
import mne
import os
from glob import glob

In [4]:
import hypyp

In [5]:
print(hypyp.__version__)

0.4.0b2


In [6]:
import mne

In [7]:
print(mne.__version__)

0.24.0


In [8]:
from hypyp.fnirs_tools import load_fnirs
from hypyp.fnirs_tools import make_fnirs_montage
from hypyp.fnirs_tools import fnirs_epoch
from hypyp.fnirs_tools import fnirs_montage_ui

In [9]:
# Some configuration variables
study_dir = "/data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean"
participant_num_len = 4 # default length of participant numbers
ex_subs = [] # any subjects to not include in the dataset

In [10]:
session_dirs = [d for d in glob(study_dir+"/*/V1/*/*") \
    if os.path.basename(os.path.split(os.path.split(d)[1])[1]) not in ex_subs]

# subjects = list(set([os.path.basename(d)[:participant_num_len] for d in session_dirs]))

In [11]:
scans = []

for dyad_dir in list(set([os.path.split(ses)[0] for ses in session_dirs]))[1:2]:
#     try:
    sub1 = os.path.join(dyad_dir, "Subject1")
    sub2 = os.path.join(dyad_dir, "Subject2")
    
    print(sub1)
    print(sub2)

    fnirs_participant_1 = load_fnirs(sub1, sub2, attr=None, preload=False, verbose=None)[0]
    fnirs_participant_2 = load_fnirs(sub1, sub2, attr=None, preload=False, verbose=None)[1]

    scans.append(fnirs_participant_1)
    scans.append(fnirs_participant_2)

#     except:
#         print("Not both subject dirs available:", dyad_dir)
#         continue

/data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean/7036/V1/2020-02-20_001/Subject1
/data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean/7036/V1/2020-02-20_001/Subject2
Loading /data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean/7036/V1/2020-02-20_001/Subject1


TypeError: expected dtype object, got 'numpy.dtype[float64]'

In [18]:
len(scans)

0